# Bestimmung der Landessprache

In den folgenden Kapiteln werden wir uns mit der *linguistischen Analyse* beschäftigen. Dort geht es um Satzbau, Wortarten usw. Die Analysemethoden bzw. die dazu verwendeten Modelle sind abhängig von der verwendeten Landessprache.

Nicht alle Dokumente eines *Korpus* müssen unbedingt in der gleichen Landessprache verfasst sein. Mehrsprachige Auswertungen sind zwar möglich, aber an sehr vielen Stellen musst du vorsichtig sein (z.B. beim Zählen der Worthäufigkeiten). Deswegen solltest du im ersten Schritt für jedes Dokument die Landessprache bestimmen.

## Nutzung von fastText Embeddings

Noch vor relativ kurzer Zeit war die Bestimmung der Landessprache eines Dokuments wirklich *schwierig*. Es gab fast schon esoterische Ansätze (ZIP-Dateien mit Texte in verschiedenen Sprachen bauen und Größen vergleichen!), die aber auch nicht richtig funktioniert haben.

In der Zwischenzeit ist das Problem größtenteils gelöst - und das auch noch sehr elegant. Mithilfe von *Word Embeddings* (die du später noch genauer kennenlernen wirst).

Dankenswerterweise gibt es schon vortrainierte Modelle, die du einfach herunterladen kannst:

In [ ]:
import requests
model_filename = "lid.176.ftz"
r = requests.get(f"https://dl.fbaipublicfiles.com/fasttext/supervised-models/{model_filename}")
open(model_filename, 'wb').write(r.content)

Jetzt kannst du das `fasttext`-Modul importieren und das Modell laden:

In [ ]:
!pip install fasttext

In [ ]:
import fasttext
lang_model = fasttext.load_model(model_filename)

Probiere es mit einigen Texten aus:

In [ ]:
lang_model.predict("comment tu t'appelles?")

In [ ]:
lang_model.predict("what's your name?")

In [ ]:
lang_model.predict("wie heißt du?")

Als Ergebnis erhältst du jeweils eine Liste der erkannten Sprache und eine mit deren Wahrscheinlichkeiten. In der Standardeinstellung liefert dir `predict` immer nur einen einzigen Vorschlag, das kannst du aber umstellen:

In [ ]:
lang_model.predict("comment tu t'appelles?", k=5)

Eine Liste der [ISO-639-1-Sprachabkürzungen](https://de.wikipedia.org/wiki/Liste_der_ISO-639-1-Codes) hilft dir bei der Interpretation. *Latein* kannst du z.B. in den allermeisten Fällen eher ausschließen.

Die Wahrscheinlichkeiten für die Sprachen addieren sich zu `1.0`. Wenn z.B. der erste Vorschlag doppelt so wahrscheinlich ist wie der zweite, kannst du diesen auch akzeptieren - selbst wenn einer eine *relativ kleine Wahrscheinlichkeit hat*. 

## Benchmarking

Ein sehr gutes Beispiel für Texte in unterschiedlichen Sprachen findet sich auf http://crism.maden.org/dunno.html. Dort steht der Satz "I don’t know. I only work here." in 63 Sprachen zur Verfügung. Zuerst lädst du die Datei herunter und übergibst sie an `BeautifulSoup`:

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(requests.get("http://crism.maden.org/dunno.html").content)

Mithilfe der bisher erklärten Technologien kannst du den Text inkl. der richtigen Antworten aus dem HTML extrahieren. Die Texte stehen alle in `<p class="translation">`-Tags, die richtigen Antworten in dem Attribut `lang`. Eine Vorhersage wird als richtig gewertet, wenn die erste Sprache doppelt so wahrscheinlich wie die zweite ist. Sprachen, die mit `x-` beginnen, werden ausgeschlossen. Englisch hat kein `lang`-Attribute, das kannst du durch die Exception abfangen:

In [ ]:
import regex as re
ok = not_ok = 0
for p in soup.select("p.translation"):
    text = p.text.strip().replace("\n", " ")
    try:
        correct = re.sub(r"\-.*", "", p.attrs["lang"])
    except:
        correct = "en"
    if correct != 'x':
        l = lang_model.predict(text, k=2)
        # Hauptsprache doppelt so wahrscheinlich wie zweite Wahl?
        if l[1][0] > 2*l[1][1]:
            predict = l[0][0].replace("__label__", "")
            if predict == correct:
                ok += 1
            else:
                print(f"Fehler bei '{text}'")
                print(f"sollte {correct} sein, {predict} vorhergesagt")
                not_ok += 1

print(f"ok = {ok}, not ok = {not_ok}")

Kurz zur Bewertung der Ergebnisse:

* Walisisch `cy` wurde als Westfriesisch `fy` klassifiziert
* `dk` ist im Text falsch kodiert, das Ergebnis ist richtig
* Farsi `fa` wurde als Arabisch `ar` klassifiziert
* Galizisch `gl` wurde als Portugiesisch `pt` klassifiziert
* `got` und `ceb` sind keine gültigen Abkürzungen
* Javanisch `jv` wurde als Englisch `en` klassifiziert
* Norwegisch `nb` wurde als Dänisch `da` klassifiziert
* Sächsisch `nds` wurde als Deutsch `de` klassifiziert
* `se` ist im Text falsch kodiert, das Ergebnis ist richtig

Abgesehen von etwas unüblichen Sprachen sind eigentlich nur die Fehlklassifikation von *Farsi* und evtl. *Javanisch* problematisch. `fasttext` arbeitet also sehr zuverlässig in der Sprachklassifikation.

## Sprachen vor der Analyse bestimmen!

Wenn du dir nicht ganz sicher bist, ob alle Dokumente deines Korpus in der gleichen Sprache verfasst sind (besonderes bei User Generated Content), solltest du die Erkennung der Landessprache immer am Anfang deiner Analyse durchführen.